In [ ]:
pip install pycocotools opencv-python numpy

In [ ]:
from pycocotools.coco import COCO
import json
import os

json_path = "/content/drive/MyDrive/DIP_Grp_5/LIVECell_dataset_2021/annotations/LIVECell/livecell_coco_train.json"

# Load JSON
with open(json_path, "r") as f:
    data = json.load(f)

# If annotations are a dict (not a list), convert them to a list
if isinstance(data.get("annotations"), dict):
    data["annotations"] = list(data["annotations"].values())

# Same for images (rare but safe)
if isinstance(data.get("images"), dict):
    data["images"] = list(data["images"].values())

# If categories are missing (sometimes happens), create dummy one
if "categories" not in data:
    data["categories"] = [{"id": 1, "name": "cell"}]

# Save fixed version
fixed_json_path = json_path.replace(".json", "_fixed.json")
with open(fixed_json_path, "w") as f:
    json.dump(data, f)

# Now load with COCO
coco = COCO(fixed_json_path)

loading annotations into memory...
Done (t=35.01s)
creating index...
index created!


In [ ]:
import numpy as np
import cv2
from tqdm import tqdm

output_dir = "/content/drive/MyDrive/DIP_Grp_5/LIVECell_dataset_2021/annotations/LIVECell/livecell_coco_train_converted_tif"
os.makedirs(output_dir, exist_ok=True)

for img_info in tqdm(coco.dataset['images']):
    img_id = img_info['id']
    img_name = os.path.splitext(img_info['file_name'])[0]
    ann_ids = coco.getAnnIds(imgIds=img_id)
    anns = coco.loadAnns(ann_ids)

    mask = np.zeros((img_info['height'], img_info['width']), dtype=np.uint8)
    for ann in anns:
        mask = np.maximum(mask, coco.annToMask(ann) * 255)

    cv2.imwrite(os.path.join(output_dir, f"{img_name}.tif"), mask)


100%|██████████| 3253/3253 [19:10<00:00,  2.83it/s]
